# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [2]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv


In [3]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [4]:
# Import the CSVs to Pandas DataFrames
file_path = Path("sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations


In [8]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    yearly_housing = sfo_data.groupby('year')['housing_units'].mean()
    housing_plot = yearly_housing.hvplot.bar(x='year', y='housing_units', ylim=(370000, 390000)).opts(yformatter="%.0f", title="Housing Units in San Francisco from 2010 - 2016", xlabel='Year', ylabel='Number of Housing Units')
    return housing_plot
    
def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    yearly_pricing = sfo_data.groupby('year')['sale_price_sqr_foot','gross_rent'].mean()
    rent_plot = yearly_pricing.hvplot.line(x='year', y='gross_rent').opts(yformatter="$%.2f", title="Average Gross Rent in San Francisco", xlabel='Year', ylabel='Avg. Gross Rent')
    return rent_plot

def average_sales_price():
    """Average Sales Price Per Year."""
    yearly_pricing = sfo_data.groupby('year')['sale_price_sqr_foot','gross_rent'].mean()
    sales_plot = yearly_pricing.hvplot.line(x='year', y='sale_price_sqr_foot').opts(yformatter="$%.2f", title="Average Sale Price per Square Foot in San Francisco", xlabel='Year', ylabel='Avg. Sale Price per Square Foot')
    return sales_plot

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    price_neighborhood = sfo_data.groupby(['year', 'neighborhood'])
    price_neighborhooddf = price_neighborhood.mean()
    neighborhood_plot = price_neighborhooddf.hvplot.line(x='year', y='sale_price_sqr_foot', groupby='neighborhood', ylim=(0, 1500)).opts(yformatter="$%.2f", xlabel='Year', ylabel='Avg. Sale Price per Square Foot')
    return neighborhood_plot

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    neighborhood_means = sfo_data.groupby('neighborhood').mean()
    top_10 = neighborhood_means.nlargest(10, 'sale_price_sqr_foot')
    top_10_plot = top_10.hvplot.bar(x='neighborhood', y='sale_price_sqr_foot', title='Top 10 Most Expensive Neighborhoods in SFO', rot=90, ylim=(0, 1500)).opts(yformatter="$%.2f",height=500, xlabel='Neighborhood', ylabel='Avg. Sale Price per Square Foot')
    return top_10_plot

def parallel_coordinates():
    """Parallel Coordinates Plot."""
    neighborhood_means = sfo_data.groupby('neighborhood').mean()
    top_10 = neighborhood_means.nlargest(10, 'sale_price_sqr_foot')
    top_10 = top_10.reset_index()
    parallel_fig = px.parallel_coordinates(top_10, color="sale_price_sqr_foot", labels={"sale_price_sqr_foot": "Sale price per square foot",
                "housing_units": "Number of Housing Units", "gross_rent": "Gross Rent"},  width=1500)
    return parallel_fig

def parallel_categories():
    """Parallel Categories Plot."""
    neighborhood_means = sfo_data.groupby('neighborhood').mean()
    top_10 = neighborhood_means.nlargest(10, 'sale_price_sqr_foot')
    top_10['sale_price_sqr_foot'] = top_10['sale_price_sqr_foot'].round(2)
    top_10['gross_rent'] = top_10['gross_rent'].round(2)
    top_10['housing_units'] = top_10['housing_units'].round(0)
    top_10 = top_10.reset_index()
    fig_categories = px.parallel_categories(top_10, dimensions=['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'], color="sale_price_sqr_foot", labels={"neighborhood":"Neighborhood", "sale_price_sqr_foot": "Sale price per square foot",
                "housing_units": "Number of Housing Units", "gross_rent": "Gross Rent"}, width=1500)
    return fig_categories


def neighborhood_map():
    """Neighborhood Map"""
    neighborhood_means = sfo_data.groupby('neighborhood').mean()
    neighborhood_means = neighborhood_means.reset_index()
    neighborhoodlatlon = (pd.concat([neighborhood_means, df_neighborhood_locations], axis='columns').reset_index())
    neighborhood_map = px.scatter_mapbox(
        neighborhoodlatlon,
        lat="Lat",
        lon="Lon",
        color="gross_rent",
        size="sale_price_sqr_foot",
        hover_data={"neighborhood":True, "gross_rent":True,"sale_price_sqr_foot":True,"Lat":False,"Lon":False},
        title="Average Sale Price per Square Foot and Gross Rent in San Francisco"
    )
    return neighborhood_map



## Panel Dashboard

In [9]:
# Create panels to structure the layout of the dashboard

welcome_plot = pn.Column(
    "This dashboard presents a visual analysis of historical prices of housing units, sale price per square foot and gross rent in San Francisco, California from 2010 to 2016.  You can navigate through the tabs above to explore more details about the evolution of the real estate market in The Golden City across these years.",
    neighborhood_map()
)

yearly_market_analysis = pn.Column(
    housing_units_per_year(),
    average_gross_rent(),
    average_sales_price()
)

neighborhood_analysis = pn.Column(
    average_price_by_neighborhood(),
    top_most_expensive_neighborhoods()
)

parallel_plot_analysis = pn.Column(
    parallel_coordinates(),
    parallel_categories()
)

# Create tabs

panel = pn.Tabs(
    ("Welcome", welcome_plot), ("Yearly Market Analysis", yearly_market_analysis), 
    ("Neighborhood Analysis", neighborhood_analysis), ("Parallel Plot Analysis", parallel_plot_analysis)
)

## Serve the Panel Dashboard

In [10]:
panel.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Column
        [0] HoloViews(Bars)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
    [2] Column
        [0] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [1] HoloViews(Bars)
    [3] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)